In [61]:
import pandas as pd
from dash import Dash, html, dcc, callback, Output, Input
import censusgeocode as cg
import plotly.figure_factory as ff
from urllib.request import urlopen
import json
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from sklearn.linear_model import LinearRegression
from Partisian import get_state_part_score
from Life import get_age_life_fig, get_state_age_life_exp
from sklearn.neighbors import KNeighborsClassifier
from plotly.subplots import make_subplots
import math
from LifeData import LifeData
import Partisian
from sklearn.linear_model import LinearRegression
import addfips


In [62]:
states = { 'AK': 'Alaska', 'AL': 'Alabama', 'AR': 'Arkansas', 'AS': 'American Samoa', 'AZ': 'Arizona', 'CA': 'California', 'CO': 'Colorado', 'CT': 'Connecticut', 'DC': 'District of Columbia', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia', 'GU': 'Guam', 'HI': 'Hawaii', 'IA': 'Iowa', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'MA': 'Massachusetts', 'MD': 'Maryland', 'ME': 'Maine', 'MI': 'Michigan', 'MN': 'Minnesota', 'MO': 'Missouri', 'MP': 'Northern Mariana Islands', 'MS': 'Mississippi', 'MT': 'Montana', 'NA': 'National', 'NC': 'North Carolina', 'ND': 'North Dakota', 'NE': 'Nebraska', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NV': 'Nevada', 'NY': 'New York', 'OH': 'Ohio', 'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'PR': 'Puerto Rico', 'RI': 'Rhode Island', 'SC': 'South Carolina', 'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VA': 'Virginia', 'VI': 'Virgin Islands', 'VT': 'Vermont', 'WA': 'Washington', 'WI': 'Wisconsin', 'WV': 'West Virginia', 'WY': 'Wyoming'
}


states = {v: k for k, v in states.items()}
states


{'Alaska': 'AK',
 'Alabama': 'AL',
 'Arkansas': 'AR',
 'American Samoa': 'AS',
 'Arizona': 'AZ',
 'California': 'CA',
 'Colorado': 'CO',
 'Connecticut': 'CT',
 'District of Columbia': 'DC',
 'Delaware': 'DE',
 'Florida': 'FL',
 'Georgia': 'GA',
 'Guam': 'GU',
 'Hawaii': 'HI',
 'Iowa': 'IA',
 'Idaho': 'ID',
 'Illinois': 'IL',
 'Indiana': 'IN',
 'Kansas': 'KS',
 'Kentucky': 'KY',
 'Louisiana': 'LA',
 'Massachusetts': 'MA',
 'Maryland': 'MD',
 'Maine': 'ME',
 'Michigan': 'MI',
 'Minnesota': 'MN',
 'Missouri': 'MO',
 'Northern Mariana Islands': 'MP',
 'Mississippi': 'MS',
 'Montana': 'MT',
 'National': 'NA',
 'North Carolina': 'NC',
 'North Dakota': 'ND',
 'Nebraska': 'NE',
 'New Hampshire': 'NH',
 'New Jersey': 'NJ',
 'New Mexico': 'NM',
 'Nevada': 'NV',
 'New York': 'NY',
 'Ohio': 'OH',
 'Oklahoma': 'OK',
 'Oregon': 'OR',
 'Pennsylvania': 'PA',
 'Puerto Rico': 'PR',
 'Rhode Island': 'RI',
 'South Carolina': 'SC',
 'South Dakota': 'SD',
 'Tennessee': 'TN',
 'Texas': 'TX',
 'Utah': 'UT',
 

In [63]:
def translate_ICD10(x, conversion):
    try:
        return conversion[x]
    except:
        return x
    
    



def clean_state_data(df):
    af = addfips.AddFIPS()
    try:
        df = df.dropna(subset=["State","ICD-10 113 Cause List Code", "Population"])
    except: 
        df = df.dropna(subset=["State", "Population"])
    #df = df[df["Crude Rate"] != "Unreliable"]
    df.Population = df.Population.astype(float)
    df["Rate"] = (df["Deaths"]/df["Population"])*100000
    df["Rate"] = df["Rate"].round(2)
    df["fips"] = df.apply(lambda x: af.get_state_fips(x["State"]), axis=1)
    df["Year"] = 2019
    df["State_Abv"] = df.apply(lambda x: states[x["State"]], axis=1)
    df = df.sort_values(by=["Rate"], ascending=False)
    df.drop(columns=["Notes", "State Code", ], inplace=True)
   
    try:
        f = open("data/Life/Deaths/ICD10Translation.json")
        translation = json.load(f)
        df["ICD-10 Common"] = df["ICD-10 113 Cause List"].apply(lambda x: translate_ICD10(x, translation))
        df.rename(columns={"ICD-10 Common":"Cause Of Death"}, inplace=True)
    except:
        None
    return df
    


def top_n(df, n):
    #Filter out low occuring chapters
    df = df.groupby("ICD-10 113 Cause List Code").filter(lambda x: len(x)>1)
    df = df.sort_values(by = ["Rate"], ascending = False)
    df = df.head(n).copy()
    top_codes = list(df["ICD-10 113 Cause List Code"].unique())
    df = df[df["ICD-10 113 Cause List Code"].isin(top_codes)]
    return df


firearms = pd.read_csv(r"data/Life/Deaths/FireArmState.txt", delimiter="	", na_values = ['Not Applicable'])
df = pd.read_csv(r"data/Life/Deaths/StateDeathsAge.txt", delimiter="	", na_values = ['Not Applicable'])
state_pop = pd.read_csv(r"data/Life/Deaths/State_Pop.txt", delimiter="	", na_values = ['Not Applicable'])
yearly_pop = pd.read_csv(r"data/Life/Deaths/2019State1YearPop.txt", delimiter="	", na_values = ['Not Applicable'])
state_deaths = pd.read_csv(r"data/Life/Deaths/2019StateDeaths.txt", delimiter="	", na_values = ['Not Applicable'])


df = clean_state_data(df)
firearms = clean_state_data(firearms)
state_pop = clean_state_data(state_pop)
yearly_pop = clean_state_data(yearly_pop)
state_deaths = clean_state_data(state_deaths)
state_pop["Total Pop"] = state_pop["Population"]

#top_df = top_n(df, 5)
#county = df.groupby(by=["2006 Urbanization Code", "ICD Sub-Chapter"]).mean().reset_index().sort_values(by=["Rate"], ascending=False)
state_pop.sample(5)
firearms = firearms.merge(state_pop[["State", "Total Pop"]], left_on="State",right_on="State")

In [64]:
part = Partisian.get_state_part_score()
final = pd.merge(part, firearms, on="fips")
final[final["State_x"] == "VT"]
state_totals = final.groupby(by=["fips","State_y"]).mean().reset_index().sort_values(by="Rate")
state_totals["Rate"] = (state_totals["Deaths"]/state_totals["Total Pop"])*100000

C:\Users\zeusg\AppData\Local\Temp/ipykernel_40072/127021580.py:4: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [65]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=state_totals["Part_Score"], y=state_totals["Rate"], mode = "markers",  hovertext=state_totals["State_y"], marker=dict(size=state_totals["Total Pop"]/1000000, colorscale='bluered', color=state_totals["Part_Score"])))

In [66]:
state_totals.sort_values("Rate", ascending=False, inplace=True)
fig = go.Figure()
fig.add_trace(go.Bar(x=state_totals["State_y"], y=state_totals["Rate"], name="Rate"))
fig.show()

In [67]:
state_totals[state_totals["State_y"] == "Vermont"]

,fips,State_y,Part_Score,Year_x,Deaths,Population,Rate,Year_y,Total Pop
43,50,Vermont,-0.4,2024.0,137.0,6904365.5,21.955515,2019.0,623989.0


In [68]:
state_totals

,fips,State_y,Part_Score,Year_x,Deaths,Population,Rate,Year_y,Total Pop
43,50,Vermont,-0.400000,2024.0,137.000000,6.904366e+06,21.955515,2019.0,623989.0
32,38,North Dakota,1.000000,2024.0,87.750000,3.945920e+06,11.514811,2019.0,762062.0
48,56,Wyoming,0.866667,2024.0,50.300000,1.349089e+06,8.691010,2019.0,578759.0
17,22,Louisiana,0.333333,2024.0,307.130952,2.346557e+06,6.606680,2019.0,4648794.0
23,28,Mississippi,0.800000,2024.0,181.063291,1.558334e+06,6.083811,2019.0,2976149.0
7,10,Delaware,-1.000000,2024.0,56.611111,7.961586e+05,5.813638,2019.0,973764.0
1,02,Alaska,0.466667,2024.0,41.755556,5.553994e+05,5.707859,2019.0,731545.0
0,01,Alabama,0.866667,2024.0,251.875000,2.504941e+06,5.136967,2019.0,4903185.0
29,35,New Mexico,-0.466667,2024.0,106.769231,1.217324e+06,5.091938,2019.0,2096829.0
19,24,Maryland,-0.533333,2024.0,295.526316,3.157371e+06,4.888223,2019.0,6045680.0


In [69]:
12/133915.0

8.960908038681252e-05

In [70]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=state_totals["Population"], y=state_totals["Rate"], mode = "markers", hovertext=state_totals["State_y"], marker=dict(colorscale='bluered', color=state_totals["Population"])))

In [71]:
usa_total = yearly_pop.groupby("Single-Year Ages").sum().reset_index()
usa_total["age"] = usa_total["Single-Year Ages"].str.replace("< 1 year","0").str.split(" ").str[0].astype(int)
usa_total.sort_values("age", inplace=True)
usa_total["Rate"] = (usa_total["Deaths"]/usa_total["Population"])*100000
usa_total

C:\Users\zeusg\AppData\Local\Temp/ipykernel_40072/196067746.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,Single-Year Ages,Deaths,Population,Rate,Year,age
84,< 1 year,20921.0,3783052.0,553.019097,102969,0
0,1 year,1401.0,3621711.0,38.683374,74703,1
11,2 years,835.0,3463061.0,24.111617,62589,2
22,3 years,596.0,3228814.0,18.458790,48456,3
33,4 years,488.0,3201427.0,15.243202,48456,4
...,...,...,...,...,...,...
78,80 years,68349.0,1475278.0,4632.957314,102969,80
79,81 years,70621.0,1381641.0,5111.385664,102969,81
80,82 years,71124.0,1241341.0,5729.610155,102969,82
81,83 years,73524.0,1151190.0,6386.782373,102969,83


In [72]:
fig = go.Figure()
fig.add_trace(go.Bar(x=usa_total["age"], y=usa_total["Population"]))
fig.add_trace(go.Bar(x=usa_total["age"], y=usa_total["Rate"]))

In [73]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=usa_total["age"], y=usa_total["Rate"], mode="lines"))

In [74]:
df["Ten-Year Age Groups Code"].unique()

array(['85+', '75-84', '65-74', '55-64', '1', '45-54', '35-44', '25-34',
       '15-24', '1-4', '5-14'], dtype=object)

In [75]:
child = state_deaths[state_deaths["Ten-Year Age Groups Code"] == '5-14']
part = Partisian.get_state_part_score()
child = pd.merge(child, part, left_on="State_Abv", right_on="State")


In [76]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=child['Part_Score'], y=child['Rate'],mode='markers',name='Child'))

In [77]:
fig = go.Figure()
fig.add_trace(go.Bar(x=child["State_Abv"], y=child["Rate"], marker=dict(color=child["Part_Score"], colorscale="Bluered" ), name="Child"))

In [78]:
child

,State_x,Ten-Year Age Groups,Ten-Year Age Groups Code,Deaths,Population,Crude Rate,Rate,fips_x,Year_x,State_Abv,State_y,fips_y,Part_Score,Year_y,category
0,Mississippi,5-14 years,5-14,98.0,395659.0,24.8,24.77,28,2019,MS,MS,28,0.800000,2024,Right
1,Idaho,5-14 years,5-14,59.0,254788.0,23.2,23.16,16,2019,ID,ID,16,1.000000,2024,Right
2,Alaska,5-14 years,5-14,22.0,100662.0,21.9,21.86,02,2019,AK,AK,02,0.466667,2024,Right
3,Alabama,5-14 years,5-14,126.0,608466.0,20.7,20.71,01,2019,AL,AL,01,0.866667,2024,Right
4,Wyoming,5-14 years,5-14,15.0,76734.0,Unreliable,19.55,56,2019,WY,WY,56,0.866667,2024,Right
5,Oklahoma,5-14 years,5-14,101.0,538048.0,18.8,18.77,40,2019,OK,OK,40,0.866667,2024,Right
6,Louisiana,5-14 years,5-14,113.0,607936.0,18.6,18.59,22,2019,LA,LA,22,0.333333,2024,Right
7,Montana,5-14 years,5-14,24.0,129305.0,18.6,18.56,30,2019,MT,MT,30,0.200000,2024,Center
8,Arkansas,5-14 years,5-14,70.0,393213.0,17.8,17.80,05,2019,AR,AR,05,0.333333,2024,Right
9,Kentucky,5-14 years,5-14,97.0,560417.0,17.3,17.31,21,2019,KY,KY,21,0.200000,2024,Center


In [79]:
import GraphGenerator
def get_age_death(df, age):
    age_df = df[df["Ten-Year Age Groups Code"] == age]
    part = Partisian.get_state_part_score()
    age_df = pd.merge(age_df, part, left_on="State_Abv", right_on="State")
    X = age_df['Part_Score'].values.reshape(-1,1)
    y = age_df['Rate']
    
    reg = LinearRegression().fit(X, y, sample_weight=age_df['Population'])
    x_range = np.linspace(X.min(), X.max())
    y = reg.predict(x_range.reshape(-1,1))
    
    fig = make_subplots(rows=1, cols=2, vertical_spacing= 0.01, horizontal_spacing= 0.01,  subplot_titles=("",""), column_widths=[0.8, 0.2], shared_yaxes=True)
    
    fig.add_trace(go.Scatter(x=age_df['Part_Score'], y=age_df['Rate'],mode='markers', marker=dict(color=age_df["Part_Score"], colorscale="Bluered", size=age_df["Population"]/(age_df["Population"].max()/50)), name='Child', hovertext=age_df["State_x"]), row=1, col=1)
    
    hist_x = []
    for index, row in age_df.iterrows():
        hist_x.extend([row["Rate"] for i in range(int(round(row["Population"]/1000)))])
        row["Rate"]
    #fig.add_trace(go.Violin(y=hist_x, side="positive", name="Death Rate Distribution"), row=1, col=2)
    fig.add_trace(go.Violin(y=age_df['Rate'], side="positive",  name="Death Rate Distribution"), row=1, col=2)
    
    
    fig.add_trace(go.Scatter(x=x_range, y=y, mode='lines',  line=dict(color="green", dash="dash"), name='Best Fit Line'), row=1, col=1)
    
    fig.update_layout(title="Partisanship Score vs. Death Rate for " + age + " Year Olds", xaxis1_title="Partisanship Score", yaxis1_title="Death Rate (per 100,000)")
    fig.update_yaxes(showticklabels=False, row=1, col=2)
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=False)
    fig.update_layout(height=300,width=1200, plot_bgcolor='rgba(0,0,0,0)', margin=dict(l=0, r=0, t=25, b=0))
    
    fig.show()


In [80]:
get_age_death(state_deaths,"85+")

In [81]:
for age in state_deaths["Ten-Year Age Groups Code"].unique():
    None
    #get_age_death(state_deaths,age)

In [82]:

fig = go.Figure()

for age in state_deaths["Ten-Year Age Groups Code"].unique():
    age_df = state_deaths[state_deaths["Ten-Year Age Groups Code"] == age]
    part = Partisian.get_state_part_score()
    age_df = pd.merge(age_df, part, left_on="State_Abv", right_on="State")
    fig.add_trace(go.Scatter(x=age_df['Part_Score'], y=age_df['Rate'],mode='markers',name=age))
    #fig.add_trace(go.Bar(x=age_df["State_Abv"], y=age_df["Rate"], marker=dict(color=age_df["Part_Score"], colorscale="Bluered" ), name=age))

fig.show()

In [83]:
teen = df[df["Ten-Year Age Groups Code"] == "85+"]
teen.sample(3)
teen_all = teen#[teen["ICD-10 Original"].str.contains("#") == True]
ny = teen_all[teen_all["State"] == "New York"]
ny.dropna(inplace=True, subset=["ICD-10 113 Cause List"])
ny.tail(3)

,State,ICD-10 113 Cause List,ICD-10 113 Cause List Code,Ten-Year Age Groups,Ten-Year Age Groups Code,Deaths,Population,Crude Rate,Rate,fips,Year,State_Abv,Cause Of Death
14967,New York,"Bronchitis, chronic and unspecified (J40-J42)",GR113-083,85+ years,85+,14.0,460001.0,Unreliable,3.04,36,2019,NY,"Bronchitis, chronic and unspecified (J40-J42)"
15153,New York,"Accidental exposure to smoke, fire and flames ...",GR113-121,85+ years,85+,13.0,460001.0,Unreliable,2.83,36,2019,NY,"Accidental exposure to smoke, fire and flames ..."
14875,New York,Acute and subacute endocarditis (I33),GR113-065,85+ years,85+,10.0,460001.0,Unreliable,2.17,36,2019,NY,Acute and subacute endocarditis (I33)


In [84]:
f = open("icd-10-Structure.json")
heirarchy = json.load(f)
heirarchy

f = open("icd-10-flat-Structure.json")
flat_heirarchy = json.load(f)


In [85]:
#ny.loc[15081]

In [86]:
ny["ICD-10 113 Cause List"] = pd.Categorical(ny["ICD-10 113 Cause List"], categories = list(heirarchy.keys()))
ny = ny.sort_values(by=["ICD-10 113 Cause List"])
ny["ICD-10 113 Cause List"].unique()
ny.dropna(subset=["ICD-10 113 Cause List"], inplace=True)

In [87]:
ny

,State,ICD-10 113 Cause List,ICD-10 113 Cause List Code,Ten-Year Age Groups,Ten-Year Age Groups Code,Deaths,Population,Crude Rate,Rate,fips,Year,State_Abv,Cause Of Death
14572,New York,"Certain other intestinal infections (A04,A07-A09)",GR113-003,85+ years,85+,176.0,460001.0,38.3,38.26,36,2019,NY,"Certain other intestinal infections (A04,A07-A09)"
14581,New York,#Septicemia (A40-A41),GR113-010,85+ years,85+,559.0,460001.0,121.5,121.52,36,2019,NY,Septicemia
14600,New York,Other and unspecified infectious and parasitic...,GR113-018,85+ years,85+,196.0,460001.0,42.6,42.61,36,2019,NY,Other and unspecified infectious and parasitic...
14610,New York,#Malignant neoplasms (C00-C97),GR113-019,85+ years,85+,6597.0,460001.0,1434.1,1434.13,36,2019,NY,Cancer
14759,New York,"#In situ neoplasms, benign neoplasms and neopl...",GR113-044,85+ years,85+,318.0,460001.0,69.1,69.13,36,2019,NY,"#In situ neoplasms, benign neoplasms and neopl..."
14765,New York,#Anemias (D50-D64),GR113-045,85+ years,85+,94.0,460001.0,20.4,20.43,36,2019,NY,#Anemias (D50-D64)
14772,New York,#Diabetes mellitus (E10-E14),GR113-046,85+ years,85+,1153.0,460001.0,250.7,250.65,36,2019,NY,Diabetes
14776,New York,#Nutritional deficiencies (E40-E64),GR113-047,85+ years,85+,167.0,460001.0,36.3,36.30,36,2019,NY,#Nutritional deficiencies (E40-E64)
14783,New York,#Parkinson disease (G20-G21),GR113-051,85+ years,85+,733.0,460001.0,159.3,159.35,36,2019,NY,#Parkinson disease (G20-G21)
14787,New York,#Alzheimer disease (G30),GR113-052,85+ years,85+,2680.0,460001.0,582.6,582.61,36,2019,NY,Alzheimer


In [88]:
labels = ["All"]
labels.extend(list(ny["ICD-10 113 Cause List"]))

values = [0]
values.extend(list(ny.Rate))


parents = []
for i,label in enumerate(labels):
    if label == "All":
        parents.append("")
    else:
        try:
            parents = flat_heirarchy[label]["parents"]
            if len(parent) > 0:
                parents.append(parent[0])
            else:
                parents.append("")
                #values[i] = values[i]-values[parents.index(heirarchy[label])]
            
        except:
            parents.append("")  



In [89]:
test_labels = labels[:11]
test_parents = parents[:11]
test_values = values[:11]
test_values[0] = sum(test_values[1:])

In [90]:
fig =go.Figure(go.Sunburst(
    labels=labels,
    parents=parents,
    values=values,
))

fig.show()

In [91]:
flat_heirarchy["#Human immunodeficiency virus (HIV) disease (B20-B24)"]

{'common_name': 'HIV',
 'children': [],
 'parents': [],
 'ranges': {'H': [], 'B': [20, 21, 22, 23, 24]},
 'chapters': ['B'],
 'base': True}

In [92]:
df = pd.read_csv(r"data/Life/Deaths/StateDeathsAge.txt", delimiter="	", na_values = ['Not Applicable'])
df = df.dropna(subset=["State","ICD-10 113 Cause List Code", "Population"])

In [93]:
df["Ten-Year Age Groups Code"].unique()

array(['1', '55-64', '65-74', '75-84', '85+', '25-34', '35-44', '45-54',
       '5-14', '1-4', '15-24'], dtype=object)

In [94]:
age = '1'

In [95]:
f = open("icd-10-Structure.json")
heirarchy = json.load(f)
heirarchy

f = open("icd-10-flat-Structure.json")
flat_heirarchy = json.load(f)


In [96]:
age

'1'

In [131]:
def load_state_cod():
    f = open("icd-10-flat-Structure.json")
    flat_heirarchy = json.load(f)
    df_heirarchy = pd.DataFrame(flat_heirarchy)
    #rotate Matrix
    df_heirarchy = df_heirarchy.transpose()
    df_heirarchy.reset_index(inplace=True)
    df_heirarchy.rename(columns={"index":"ICD-10 113 Cause List"}, inplace=True)
    df_heirarchy

    df = pd.read_csv(r"data/Life/Deaths/StateDeathsAge.txt", delimiter="	", na_values = ['Not Applicable'])
    df = df.dropna(subset=["State","ICD-10 113 Cause List Code", "Population"]) 
    state_df= df[df["State"] == "Michigan"]
    #state_df = state_df[state_df["Ten-Year Age Groups Code"] ==age]
    state_df["Rate"] = (state_df["Deaths"] / state_df["Population"])*100000
    state_df["Rate"] = np.round(state_df["Rate"], 2)
    state_df["Child_Adj_Rate"] = state_df["Rate"]


    df_final = pd.merge(state_df, df_heirarchy, on="ICD-10 113 Cause List")
    df_final = df_final.drop(columns=["ICD-10 113 Cause List Code", "ranges", "chapters", "base", "Notes"])
    return df_final

In [132]:
def add_children(name, child, order):
    order.append(name)
    if len(child["children"]) > 0: 
        for grand_child in child["children"]:
            add_children(grand_child, flat_heirarchy[grand_child], order)
def get_depth_order(flat_heirarchy):
    order = []
    for key, value in flat_heirarchy.items():
        if len(value["parents"]) == 0:
            order.append(key)
            for children in value["children"]:
                add_children(children, flat_heirarchy[children],order)
    return order

In [133]:
def get_age_cod(df, age, all_ages=False):
    f = open("icd-10-flat-Structure.json")
    flat_heirarchy = json.load(f)
    order = get_depth_order(flat_heirarchy)
    df = df[df["Ten-Year Age Groups Code"] ==age]
    for e in range(5):
        for i, row in df.iterrows():
            children_total = 0
            children_values =[]
            if len(row["children"]) > 0:
                for child in row["children"]:
                    try:
                        children_total += df[df["ICD-10 113 Cause List"] == child]["Child_Adj_Rate"].values[0]
                        children_values.append(df[df["ICD-10 113 Cause List"] == child]["Child_Adj_Rate"].values[0])
                    except:
                        None
            df.at[i, "Child_Adj_Rate"] = max(row["Child_Adj_Rate"], children_total)
    df["ICD-10 113 Cause List"] = df["ICD-10 113 Cause List"].astype("category")
    df["ICD-10 113 Cause List"] = df["ICD-10 113 Cause List"].cat.set_categories(order, ordered=True)
    df = df.sort_values("ICD-10 113 Cause List")
    
    labels = [age]
    parents = [""]
    if all_ages:
        parents = ["All Ages"]
    values = [0]
    colors = ["white"]
    for i, row in df.iterrows():
        labels.append(row["ICD-10 113 Cause List"] + " " + str(age)[:2])
        if len(row["parents"]) > 0:
            parents.append(row["parents"][0] + " " + str(age)[:2])
        else: 
            parents.append(age)
            values[0] += row["Child_Adj_Rate"]
        values.append(row["Child_Adj_Rate"])
    return labels, parents, values, df, values[0]
    
df_final = load_state_cod()
labels, parents, values, df, total = get_age_cod(df_final, age, all_ages=True)

In [134]:
df_final

,State,State Code,ICD-10 113 Cause List,Ten-Year Age Groups,Ten-Year Age Groups Code,Deaths,Population,Crude Rate,Rate,Child_Adj_Rate,common_name,children,parents
0,Michigan,26.0,"Certain other intestinal infections (A04,A07-A09)",45-54 years,45-54,10.0,1254679.0,Unreliable,0.80,0.80,Certain other intestinal infections,[#Enterocolitis due to Clostridium difficile (...,[]
1,Michigan,26.0,"Certain other intestinal infections (A04,A07-A09)",55-64 years,55-64,21.0,1399535.0,1.5,1.50,1.50,Certain other intestinal infections,[#Enterocolitis due to Clostridium difficile (...,[]
2,Michigan,26.0,"Certain other intestinal infections (A04,A07-A09)",65-74 years,65-74,32.0,1037580.0,3.1,3.08,3.08,Certain other intestinal infections,[#Enterocolitis due to Clostridium difficile (...,[]
3,Michigan,26.0,"Certain other intestinal infections (A04,A07-A09)",75-84 years,75-84,51.0,514685.0,9.9,9.91,9.91,Certain other intestinal infections,[#Enterocolitis due to Clostridium difficile (...,[]
4,Michigan,26.0,"Certain other intestinal infections (A04,A07-A09)",85+ years,85+,61.0,213136.0,28.6,28.62,28.62,Certain other intestinal infections,[#Enterocolitis due to Clostridium difficile (...,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,Michigan,26.0,#Complications of medical and surgical care (Y...,85+ years,85+,15.0,213136.0,Unreliable,7.04,7.04,Medical Care Complications,[],[]
599,Michigan,26.0,#Enterocolitis due to Clostridium difficile (A...,55-64 years,55-64,15.0,1399535.0,Unreliable,1.07,1.07,#Enterocolitis due to Clostridium difficile,[],"[Certain other intestinal infections (A04,A07-..."
600,Michigan,26.0,#Enterocolitis due to Clostridium difficile (A...,65-74 years,65-74,21.0,1037580.0,2.0,2.02,2.02,#Enterocolitis due to Clostridium difficile,[],"[Certain other intestinal infections (A04,A07-..."
601,Michigan,26.0,#Enterocolitis due to Clostridium difficile (A...,75-84 years,75-84,34.0,514685.0,6.6,6.61,6.61,#Enterocolitis due to Clostridium difficile,[],"[Certain other intestinal infections (A04,A07-..."


In [139]:
def get_all_age_cod(df):
    all_labels = ["All Ages"]
    all_parents = [""]
    all_values = [0]
    for age in df["Ten-Year Age Groups Code"].unique():
        labels, parents, values, df_age, total = get_age_cod(df, age, True)
        all_labels.extend(labels)
        all_parents.extend(parents)
        all_values.extend(values)
        all_values[0] += total
    return all_labels, all_parents, all_values
    
final_labels, final_parents, final_values = get_all_age_cod(df_final)    

In [140]:
final_labels

['All Ages',
 '45-54',
 'Certain other intestinal infections (A04,A07-A09) 45',
 '#Septicemia (A40-A41) 45',
 '#Viral hepatitis (B15-B19) 45',
 '#Human immunodeficiency virus (HIV) disease (B20-B24) 45',
 'Other and unspecified infectious and parasitic diseases and their sequelae (A00,A05,A20-A36,A42-A44,A48-A49,A54-A79,A81-A82,A85.0-A85.1,A85.8,A86-B04,B06-B09,B25-B49,B55-B99,U07.1) 45',
 '#Malignant neoplasms (C00-C97) 45',
 'Malignant neoplasms of kidney and renal pelvis (C64-C65) 45',
 'Malignant neoplasms of liver and intrahepatic bile ducts (C22) 45',
 'Malignant neoplasms of meninges, brain and other parts of central nervous system (C70-C72) 45',
 'Malignant neoplasm of stomach (C16) 45',
 'Malignant neoplasm of cervix uteri (C53) 45',
 'Malignant neoplasm of larynx (C32) 45',
 'Malignant neoplasm of bladder (C67) 45',
 'All other and unspecified malignant neoplasms (C17,C23-C24,C26-C31,C37-C41,C44-C49,C51-C52,C57-C60,C62-C63,C66,C68-C69,C73-C80,C97) 45',
 'Malignant neoplasm of

In [141]:
fig = go.Figure(go.Treemap(
    branchvalues = "total",
    labels = final_labels,
    parents =  final_parents,
    values=final_values,
))

fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))


fig.update_layout(
    autosize=False,
    width=2100,
    height=1050,
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0,
        pad=0
    ),
)

fig.show()
fig.write_html("CauseOfDeathTest.html")

In [142]:
fig = go.Figure(go.Sunburst(
    branchvalues = "total",
    labels = final_labels,
    parents =  final_parents,
    values=final_values,
))

fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))


fig.update_layout(
    autosize=False,
    width=2100,
    height=1050,
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0,
        pad=0
    ),
)

fig.show()

#here

In [125]:
def load_state_cod(state, age_group = '75-84'):
    df = pd.read_csv(r"data/Life/Deaths/StateDeathsAge.txt", delimiter="	", na_values = ['Not Applicable'])
    df = df.dropna(subset=["State","ICD-10 113 Cause List Code", "Population"]) 
    state_df= df[df["State"] == state]
    state_df = state_df[state_df["Ten-Year Age Groups Code"] =='75-84']
    state_df["Rate"] = (state_df["Deaths"] / state_df["Population"])*100000
    state_df["Rate"] = np.round(state_df["Rate"], 2)
    state_df["Child_Adj_Rate"] = state_df["Rate"]
    
    state_df["Parent"] = ""
    state_df["Children"] = ""
    state_df["Common Name"] = ""
    
    
    f = open("icd-10-flat-Structure.json")
    flat_heirarchy = json.load(f)

    targets =[]
    sources = []
    values = []

    sources.append("")
    targets.append("All")
    values.append(0)
    values.extend(list(state_df.Rate))
    


    for i, row in state_df.iterrows():
        try:
            structure = flat_heirarchy[row["ICD-10 113 Cause List"]]
            state_df.at[i, "Common Name"] = structure["common_name"]
            targets.append(structure["common_name"])
            structure["Rate"] = row["Rate"]
            
            if len(structure["parents"])  == 1:
                parent = flat_heirarchy[structure["parents"][0]]["common_name"]
                state_df.loc[i, "Parent"] = parent
                sources.append(parent)
            else:
                state_df.loc[i, "Parent"] = "All"
                sources.append("All")
        except Exception as e:
            print(e)
        if len(structure["children"]) > 0:
            state_df.at[i, "Children"] = structure["children"]
             
    for i, source in enumerate(sources):
        if source == "":
            None
        else:
            try:
                parent_index = targets.index(source)
                values[parent_index] = max( values[parent_index] - values[i],0)
                state_df.loc[state_df['Common Name'] == targets[parent_index], 'Child_Adj_Rate'] = values[parent_index]
                
            except:
                values[i] = max(values[i],0)
                state_df.loc[state_df['Common Name'] == targets[parent_index], 'Child_Adj_Rate'] = values[i]
                
    values[0] = 5000
    size = 50
    plot_dict = {"values":values[:size], "targets":targets[:size], "parents":sources[:size]}
    return state_df, plot_dict

In [126]:
df, plot_dic = load_state_cod("Kentucky")